## 1. Background

The Variantional Bayesian (VB) approach is to optimize an approximation to the intractable posterior. However, the common mean-field approach requires analytical solutions of expectation w.r.t. the approximate posterior which is also intractable in many cases. In this paper, we introduce the Stochastic Gradient Variantional Bayes (SGVB) estimator which is used for an efficient approximation of the posterior distribution in almost any case with continuous latent variables. The main idea is to yield a simple differentiable unbiased estimator of the variational lower bound based on a reparameterizatio of the lower bound. The optimization involved in this approach uses standard stochastic gradient ascent techniques. Furthermore, we propose the Auto-Encoding VB (AEVB) algorithm to implement this approach. The SGVB estimator is used in the AEVB algorithm to optimize a recognition model that allows us to efficiently approximate the posterior inference with simple ancestral sampling, which results in an efficient learning of the model parameters without costly iterative inference schemes (such as MCMC) per datapoint. Especially, it is called the variational auto-encoder when we use a neural network for the recognition model.


## 2. Algorithm

### 2.1 Problem Setting 

1. The dataset $X = \{x^{(i)}\}^N_{i=1}$ consists of $N$ i.i.d samples of some continuous or discrete variable $x$. They are generated from some random process with an unobserved continuous random variable $z$.
2. $z^{(i)}$ is generated from some prior distribution $p_{\theta^*}(z)$.
3. $x^{(i)}$ is generated from some conditional distribution $p_{\theta^*}(x|z)$.
4. Both $p_{\theta^*}(z)$ and $p_{\theta^*}(x|z)$ are from parametric families of distributions $p_\theta(z)$ and $p_\theta(x|z)$, and their PDFs are differentiable almost everywhere w.r.t. both $\theta$ and $z$. However, we do not make the common simplifying assumptions about the marginal or posterior probabilities. 
5. $q_\phi(z|x)$ is the recognition model, an approximation of the intractable true posterior $p_\theta(z|x)$. It is not necessarily factorial and its parameter $\phi$ are not computed from some closed-form expectation.
6. In the aspect of coding theory, the unobserved $z$ is interprated as a code, $q_\phi(z|x)$ is a probabilistic encoder and $p_\theta(x|z)$ is a probabilistic decoder.

### 2.2 Method

The marginal likelihood $\log p_\theta(x^{(1)},...,x^{(N)})=\sum^N_{i=1}\log p_\theta(x^{(i)})$ can be rewritten as:

\begin{equation}
\log p_\theta(x^{(i)})=D_{KL}(q_\phi(z|x^{(i)})||p_\theta(z|x^{(i)}))+L(\theta, \phi; x^{(i)})
\end{equation}

where the first RHS term is the KL divergence of the approximate from the true posterior and it is non-negative. The second RHS term $L(\theta, \phi; x^{(i)})$ is the variational lower bound on the marginal likelihood of the datapoint $i$. Thus we have:

\begin{equation}
\log p_\theta(x^{(i)})\geq L(\theta, \phi; x^{(i)})=E_{q_\phi(z|x)}[-\log q_\phi(z|x)+\log p_\theta(x,z)]\\
=-D_{KL}(q_\phi(z|x^{(i)}))||p_\theta(z|x^{(i)}))+E_{q_\phi(z|x^{(i)})}[p_\theta(x^{(i)}|z)]
\end{equation}

The point is to differentiate and optimize the lower bound $L(\theta, \phi; x^{(i)})$ w.r.t both the variational parameters $\phi$ and generative parameters $\theta$.

With a chosen $q_\phi(z|x)$, we use a differentiable transformation $g_\phi(\epsilon, x)$ to reparameterize the random variable $\widetilde{z}\sim q_\phi(z|x)$, where $\epsilon$ is an auxiliary noise:

\begin{equation}
\widetilde{z}=g_\phi(\epsilon, x), \epsilon\sim p(\epsilon)
\end{equation}

Here we can form Monte Carlo estimates of expectations of some function $f(z)$ w.r.t $q_\phi(z|x)$ as follows:

\begin{equation}
E_{q_\phi(z|x^{(i)})}[f(z)]=E_{p(\epsilon)}[f(g_\phi (\epsilon, x^{(i)}))]\simeq\frac{1}{L}\sum^L_{l=1}f(g_\phi (\epsilon^{(l)}, x^{(i)})), \epsilon^{(l)}\sim p(\epsilon)
\end{equation}

We apply this technique to the variational lower bound and yield the generic SGVB estimator $\widetilde{L}^A(\theta, \phi; x^{(i)})\simeq L(\theta, \phi; x^{(i)})$:

\begin{equation}
\widetilde{L}^A(\theta, \phi; x^{(i)})=\frac{1}{L}\sum^L_{l=1}\log p_\theta (x^{(i)}, z^{(i, l)})-\log q_\phi(z^{(i, l)}|x^{(i)})\\
z^{(i,l)}=g_\phi (\epsilon^{(i, l)}, x^{(i)}), \epsilon^{(l)}\sim p(\epsilon)
\end{equation}

When $D_{KL}(q_\phi(z|x^{(i)})||p_\theta(z|x^{(i)}))$ can be analytically integrated, we only need to estimate the expected reconstruction error $E_{q_\phi(z|x^{(i)})}[p_\theta(x^{(i)}|z)]$ by sampling. Thus we have the second version of the SGVB estimator $\widetilde{L}^B(\theta, \phi; x^{(i)})\simeq L(\theta, \phi; x^{(i)})$:

\begin{equation}
\widetilde{L}^B(\theta, \phi; x^{(i)})=-D_{KL}(q_\phi(z|x^{(i)}))||p_\theta(z|x^{(i)}))+\frac{1}{L}\sum^L_{l=1}\log p_\theta (x^{(i)}|z^{(i, l)})\\
z^{(i,l)}=g_\phi (\epsilon^{(i, l)}, x^{(i)}), \epsilon^{(l)}\sim p(\epsilon)
\end{equation}

Actually, the KL-divergence term can be interpreted as regularizing $\phi$ and $\widetilde{L}^B(\theta, \phi; x^{(i)})$ typically has less variance than $\widetilde{L}^A(\theta, \phi; x^{(i)})$.

### 2.3 AEVB Algorithm

First, we recall the autoencoder architecture:
$$\phi: X\to F \\
\psi: F\to X \\
\phi, \psi = \arg\min||X-(\psi\circ\phi) X||^2$$

In the simplest case, where there is one hidden layer, the encoder stage of an autoencoder takes the input $x\in \mathbb{R}^d = X$ that is then mapped to the latent code $z\in\mathbb{R}^p = F$:

\begin{equation}
\mathbf{z}=\sigma(\mathbf{Wx}+\mathbf{b})
\end{equation}

where $\sigma$ is an element-wise activation function such as a sigmoid function or a rectified linear unit. $\mathbf {W}$ s a weight matrix and $\mathbf {b}$ is a bias vector. After that, the decoder stage of the autoencoder maps $\mathbf {z}$ to the reconstruction $\mathbf {x'}$ of the same shape as $\mathbf{x}$:

\begin{equation}
\mathbf{z'}=\sigma'(\mathbf{W'x'}+\mathbf{b'})
\end{equation}

where $\mathbf {\sigma '} ,\mathbf {W'}$ and $\mathbf {b'}$ for the decoder may differ in general from the corresponding $\mathbf {\sigma } ,\mathbf {W}$ and $\mathbf {b}$ for the encoder.

Autoencoders are also trained to minimise reconstruction errors:

\begin{equation}
\mathbf{L(x, x')} = ||\mathbf{x}-\mathbf{x'}||^2 = ||\mathbf{x}-\sigma'(\mathbf{W'}(\sigma(\mathbf{Wx+b}))+\mathbf{b'})||^2
\end{equation}

Now it is clear to see the connection between SGVB estimator and auto-encoders above, which is that $\widetilde{L}^A(\theta, \phi; x^{(i)})$ or $\widetilde{L}^B(\theta, \phi; x^{(i)})$ should be the objective funtion ($\mathbf{L}$ above) for optimization. Therefore, we summarize the entire algorithm (a minibatch version) as follows:

- Initialize parameters $\mathbf{\theta_0, \phi_0}$ 

$\textbf{repeat}$:  

1. $X^M$: Random minibatch of M datapoints (drawn from full dataset with the size of N)
2. $\mathbf{\epsilon}$: Random samples from noise distribution $p(\epsilon)$                                      
3. 
\fbox{Encoding: $X\to Z=g_\phi(X, \epsilon)$ and
Decoding: $Z\to \log p_\theta(X|Z)$}  $\to \widetilde{L}(\mathbf{\phi}, \mathbf{\theta}; \mathbf{X}, \mathbf{\epsilon})$ (either of the two SGVB estimators above can be used)
4. $\nabla_{\theta, \phi}\widetilde{L}^M(\mathbf{\phi}, \mathbf{\theta}; \mathbf{X^M}, \mathbf{\epsilon})$: gradient of the minibatch estimators, where $\widetilde{L}^M(\mathbf{\phi}, \mathbf{\theta}; \mathbf{X^M}, \mathbf{\epsilon})=\frac{N}{M}\sum_{i=1}^M\widetilde{L}(\mathbf{\phi}, \mathbf{\theta}; x^{(i)}, \mathbf{\epsilon})$
5. Update $\mathbf{\theta, \phi}$ using $\nabla_{\theta, \phi}\widetilde{L}^M(\mathbf{\phi}, \mathbf{\theta}; \mathbf{X}, \mathbf{\epsilon})$ (e.g. SGD)

$\textbf{until}$: convergence of parameters ($\mathbf{\theta, \phi}$) 

$\textbf{return}$: $\mathbf{\theta, \phi}$

Thus, we find that parameters $\theta, \phi$ are jointly optimized via the AEVB algorithm.
                                      

## 3. Implementation

In terms of implementing the AEVB algorithm, we use a neural network for the probabilistic encoding and decoding parts. Let the prior over the latent variables be the centered isotropic multivariate Gaussian $p_\theta(\mathbf{z})=\mathcal{N}(\mathbf{z};\mathbf{0}, \mathbf{I})$. We assume the true (but intractable) posterior takes on a ap- proximate Gaussian form with an approximately diagonal covariance. In this case, we can let the variational approximate posterior be a multivariate Gaussian with a diagonal covariance structure:

$$\log q_\phi(\mathbf{z}|\mathbf{x}^{(i)})=\log\mathcal{N}(\mathbf{z}; \mathbf{\mu}^{(i)}, \mathbf{\sigma}^{2(i)}\mathbf{I})$$

where the mean and s.d. of the approximate posterior, $\mathbf{\mu}^{(i)}$ and $\mathbf{\sigma}^{2(i)}$, are outputs of the encoding MLP (a fully-connected neural network with a single hidden layer), i.e. nonlinear functions of datapoint $x^{(i)}$ and the variational parameters $\phi$:

\begin{equation}
\log q_\phi=\log\mathcal{N}(\mathbf{z}; \mathbf{\mu}, \mathbf{\sigma}^2\mathbf{I})\\
\mathbf{\mu}=\mathbf{W_1h}+\mathbf{b_1}\\
\log\mathbf{\sigma}^2=\mathbf{W_2h}+\mathbf{b_2}\\
\mathbf{h}=\tanh(\mathbf{W_3z}+\mathbf{b_3})
\end{equation}

Then, we sample from the posterior $\mathbf{z}^{(i,l)}\sim q_\phi(\mathbf{z}|\mathbf{x}^{(i)})$ using $\mathbf{z}^{(i,l)}=g_\phi(\mathbf{x}^{(i)},\epsilon^{(l)})=\mu^{(i)}+\sigma^{(i)}\bigodot\epsilon^{(l)}$ where $\epsilon^{(l)}\sim\mathcal{N}(\mathbf{0},\mathbf{I})$. With $\bigodot$ we signify an element-wise product. Since both $p_\theta(\mathbf{z})$ and $q_\phi(\mathbf{z}|\mathbf{x})$ are Gaussian, we use $\widetilde{L}^B$ with an analytically solved KL divergence. Therefore, we derive the resulting estimator as follows:

\begin{equation}
L(\theta, \phi; \mathbf{x}^{(i)})\simeq\frac{1}{2}\sum_{j=1}^J(1+\log((\sigma_j^{(i)})^2)-(\mu_j^{(i)})^2-(\sigma_j^{(i)})^2)+\frac{1}{L}\sum_{l=1}^{L}\log p_\theta(\mathbf{x}^{(i)}|\mathbf{z}^{(i,l)})
\end{equation}

Given the type of data that we are going to work on, we use the Gaussian MLP as decoder to achieve $\log p_\theta(\mathbf{x}^{(i)}|\mathbf{z}^{(i,l)})$:

\begin{equation}
\log p_\theta=\log\mathcal{N}(\mathbf{x}; \mathbf{\mu}', \mathbf{\sigma}'^2\mathbf{I})\\
\mathbf{\mu}'=\mathbf{W_4h'}+\mathbf{b_4}\\
\log\mathbf{\sigma}'^2=\mathbf{W_5h'}+\mathbf{b_5}\\
\mathbf{h'}=\tanh(\mathbf{W_6x}+\mathbf{b_6})
\end{equation}

$\{\mathbf{W}_i, \mathbf{b}_i\}_{i=1}^6$ are the weights and biases of the MLPs for encoder and decoder. To be clear, $\mathbf{\phi}=\{\mu, \sigma\}$ and $\mathbf{\theta}=\{\mu', \sigma'\}$ are parameters to be optimized.

Therefore, we summarize the iterative process of updating $\phi$ and $\theta$ as follows:

We offer some specific settings for the implementation on images from the MNIST and Frey Face datasets: 

1. The size of minibatch $M=100$ and the size of samples per datapoint $L=1$.
2. The initialized values of parameters $\phi_0$ and $\theta_0$ are randomly sampled from $\mathcal{N}(0,0.01)$.
3. Stepsizes used for updating parameters within the algorithm are adapted by Adagrad with parameters chosen from $\{0.01, 0.02, 0.1\}$.
4. In terms of obtaining the likelihood lower bound, we trained generative models (decoders) and corresponding encoders (a.k.a. recognition models) having 500 hidden units in case of MNIST, and 200 hidden units in case of the Frey Face dataset.
5. In terms of estimating the marginal likelihood of the learned generative models with an MCMC estimator, we again used neural networks for the encoder and decoder, this time with 100 hidden units and 3 latent variables in case of MNIST dataset.